In [182]:
import pandas as pd
import random
import numpy as np
import glob
import os

startDate = "20200324"
endDate = "20200327"
readPath = r'\\192.168.10.30\Kevin_zhenyu\rawData\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
spath = []
tpath = []
for i in dataPathLs:
    tpath.append(glob.glob(os.path.join(i, "mdTradeLog_***")))
    spath.append(glob.glob(os.path.join(i, "mdLog_SH_***")))
tpath = tpath[1::2]
spath = spath[1::2]
print(tpath)
print(spath)

df = pd.DataFrame()
date = []
l1 = []
l2 = []
l3 = []
l4 = []
l5 = []
l6 = []
l7 = []
l8 = []
l9 = []
l10 = []
l11 = []
l12 = []

for i in range(2, 3):
    mdTradeLog = pd.read_csv(tpath[i][0])
    date.append(os.path.basename(tpath[i][0]).split('.')[0].split('_')[1])
    mdTradeLog = mdTradeLog[mdTradeLog['SecurityID'] >= 600000]
    mdTradeLog['volumeThisUpdate'] = np.where(mdTradeLog['ExecType'] == 'F', mdTradeLog['TradeQty'], 0)
    mdTradeLog['cum_volume'] = mdTradeLog.groupby(['SecurityID'])['volumeThisUpdate'].cumsum()
    mdTradeLog = mdTradeLog[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'cum_volume',
                             'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]
    SH = pd.read_csv(spath[i][0])[["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice"]]
    sl1 = SH["StockID"].unique()
    sl2 = mdTradeLog["SecurityID"].unique()
    sl = list(set(sl1) & set(sl2))
    SH = SH[SH["StockID"].isin(sl)]
    mdTradeLog = mdTradeLog[mdTradeLog["SecurityID"].isin(sl)]
    SH["time"] = SH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    mdTradeLog = mdTradeLog[(mdTradeLog["TransactTime"] > 93000000) & (mdTradeLog["TransactTime"] < 145655000)]
    SH = SH[(SH["time"] > 93000000) & (SH["time"] < 145655000)]
    SH = SH.rename(columns={"StockID":"SecurityID", "cum_volume": "max_cum_volume_ss"})
    re = pd.concat([mdTradeLog, SH[["sequenceNo", "SecurityID", "max_cum_volume_ss"]]]).sort_values(by=["SecurityID", "sequenceNo"])
    re["max_cum_volume_ss"] = re.groupby("SecurityID")["max_cum_volume_ss"].ffill()
    re = re[(~re["cum_volume"].isnull()) | (~re["max_cum_volume_ss"].isnull())]
    kk = re[re["cum_volume"] <= re["max_cum_volume_ss"]].groupby("SecurityID")["TradeMoney"].count().reset_index()
    kk = kk.rename(columns={"TradeMoney": "delayed count"})
    kk = pd.merge(kk, re.groupby("SecurityID")["TradeMoney"].count().reset_index(), on="SecurityID")
    kk = kk.rename(columns={"TradeMoney": "total count"})
    kk = pd.merge(kk, re[re["cum_volume"] <= re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].describe().reset_index(), on="SecurityID")
    kk = kk[["SecurityID", "delayed count", "total count", "min", "25%", "50%", "75%", "max"]]
    l1.append(kk["delayed count"].sum())
    l2.append(kk["total count"].sum())
    l3.append(kk["delayed count"].sum() / kk["total count"].sum())
    l4.append((kk["delayed count"]/ kk["total count"]).describe()["25%"])
    l5.append((kk["delayed count"]/ kk["total count"]).describe()["50%"])
    l6.append((kk["delayed count"]/ kk["total count"]).describe()["75%"])
    l7.append(re[re["cum_volume"] <= re["max_cum_volume_ss"]]["TransactTime"].describe()["25%"])
    l8.append(re[re["cum_volume"] <= re["max_cum_volume_ss"]]["TransactTime"].describe()["50%"])
    l9.append(re[re["cum_volume"] <= re["max_cum_volume_ss"]]["TransactTime"].describe()["75%"])
    l10.append(kk["25%"].describe()["50%"])
    l11.append(kk["50%"].describe()["50%"])
    l12.append(kk["75%"].describe()["50%"])
df['Date'] = date
df["delayed count"] = l1
df["total count"] = l2
df["delayed perc"] = l3
df["delayed 25p"] = l4
df["delayed 50p"] = l5
df["delayed 75p"] = l6
df["time 25p"] = l7
df["time 50p"] = l8
df["time 75p"] = l9
df["time 25p 50p"] = l10
df["time 50p 50p"] = l11
df["time 75p 50p"] = l12
for col in ["delayed perc", "delayed 25p", "delayed 50p", "delayed 75p"]:
    df[col] = df[col].apply(lambda x: '%.0f%%'%(x*100))
df

[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200324_zt_96_04_day_96data\\mdTradeLog_20200324_0838.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200325_zt_96_04_day_96data\\mdTradeLog_20200325_0838.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200326_zt_96_04_day_96data\\mdTradeLog_20200326_0848.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200327_zt_96_04_day_96data\\mdTradeLog_20200327_0833.csv']]
[['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200324_zt_96_04_day_96data\\mdLog_SH_20200324_0838.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200325_zt_96_04_day_96data\\mdLog_SH_20200325_0838.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200326_zt_96_04_day_96data\\mdLog_SH_20200326_0848.csv'], ['\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_20200327_zt_96_04_day_96data\\mdLog_SH_20200327_0833.csv']]


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Date,delayed count,total count,delayed perc,delayed 25p,delayed 50p,delayed 75p,time 25p,time 50p,time 75p,time 25p 50p,time 50p 50p,time 75p 50p
0,20200326,1642713,14730964,11%,10%,17%,27%,93118050.0,103631120.0,133005130.0,95103076.25,110610605.0,134306140.0


In [65]:
for col in ["time 25p", "time 50p", "time 75p", "time 25p 50p", "time 50p 50p", "time 75p 50p"]:
    df[col] = df[col].astype("int")
df

,Date,delayed count,total count,delayed perc,delayed 25p,delayed 50p,delayed 75p,time 25p,time 50p,time 75p,time 25p 50p,time 50p 50p,time 75p 50p
0,20200324,898631,915234,98%,98%,99%,100%,93017920,93105470,93156510,93020340,93105280,93155580
1,20200325,1138243,1153800,99%,99%,99%,100%,93012030,93056560,93151380,93013090,93058580,93150640
2,20200326,482908,760607,63%,58%,65%,72%,93005910,93033320,93105860,93007570,93036470,93106600
3,20200327,515292,739738,70%,66%,73%,78%,93006820,93034310,93108450,93007770,93035520,93107467


In [189]:
r0 = re[re["cum_volume"] > re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].apply(lambda x: x.describe([0.01])[["min"]]).reset_index()
r1 = re[re["cum_volume"] > re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].apply(lambda x: x.describe([0.01])[["1%"]]).reset_index()
r2 = re[re["cum_volume"] > re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].apply(lambda x: x.describe([0.02])[["2%"]]).reset_index()
r3 = re[re["cum_volume"] > re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].apply(lambda x: x.describe([0.03])[["3%"]]).reset_index()
r4 = re[re["cum_volume"] > re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].apply(lambda x: x.describe([0.04])[["4%"]]).reset_index()
r5 = re[re["cum_volume"] > re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].apply(lambda x: x.describe([0.05])[["5%"]]).reset_index()
r6 = re[re["cum_volume"] > re["max_cum_volume_ss"]].groupby("SecurityID")["TransactTime"].apply(lambda x: x.describe([0.1])[["10%"]]).reset_index()
r = pd.merge(r0, r1, on=["SecurityID"])
r = pd.merge(r, r2, on=["SecurityID"])
r = pd.merge(r, r3, on=["SecurityID"])
r = pd.merge(r, r4, on=["SecurityID"])
r = pd.merge(r, r5, on=["SecurityID"])
r = pd.merge(r, r6, on=["SecurityID"])
r = pd.merge(r, re.groupby("SecurityID")["TransactTime"].min().reset_index(), on=["SecurityID"])
display(r)
r["TransactTime_x"].iloc[:, 0].describe([0.01, 0.05, 0.06, 0.07, 0.071, 0.072, 0.073, 0.074, 0.075, 0.08, 0.09, 0.1])

,SecurityID,level_1_x,TransactTime_x,level_1_y,TransactTime_y,level_1_x,TransactTime_x,level_1_y,TransactTime_y,level_1_x,TransactTime_x,level_1_y,TransactTime_y,level_1,TransactTime_x,TransactTime_y
0,600000,min,93000580.0,1%,93319403.5,2%,93441310.0,3%,93606512.0,4%,93756930.0,5%,94002110.0,10%,95007690.0,93000460.0
1,600004,min,93136100.0,1%,93213510.0,2%,93313400.0,3%,93439602.0,4%,93540620.0,5%,93628110.0,10%,94507480.0,93000710.0
2,600006,min,93131860.0,1%,93205922.7,2%,93246410.0,3%,93415238.0,4%,93440680.0,5%,93528550.0,10%,94050490.0,93000890.0
3,600007,min,93140420.0,1%,93254932.6,2%,93345205.6,3%,93537608.6,4%,93609275.2,5%,93696995.0,10%,94528116.0,93001480.0
4,600008,min,93138700.0,1%,93301810.0,2%,93430693.2,3%,93511500.0,4%,93625875.2,5%,93628740.0,10%,93727700.0,93000520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581,688388,min,93137820.0,1%,93201297.2,2%,93218920.0,3%,93233840.0,4%,93340019.6,5%,93414320.0,10%,93840964.0,93000630.0
1582,688389,min,93143970.0,1%,93224996.3,2%,93355020.0,3%,93559360.0,4%,93658280.0,5%,93939173.5,10%,95150802.0,93000950.0
1583,688396,min,93132870.0,1%,93214137.6,2%,93251555.6,3%,93323550.0,4%,93379412.8,5%,93442040.0,10%,94006214.0,93000490.0
1584,688398,min,93157830.0,1%,93256103.0,2%,93328900.0,3%,93339275.0,4%,93428714.0,5%,93516540.0,10%,93908190.0,93000610.0


count    1.586000e+03
mean     9.311923e+07
std      6.048283e+04
min      9.300058e+07
1%       9.300058e+07
5%       9.300058e+07
6%       9.300058e+07
7%       9.300072e+07
7.1%     9.300072e+07
7.2%     9.300072e+07
7.3%     9.300072e+07
7.4%     9.300072e+07
7.5%     9.300072e+07
8%       9.300072e+07
9%       9.300084e+07
10%      9.300084e+07
50%      9.313678e+07
max      9.393832e+07
Name: TransactTime_x, dtype: float64

In [209]:
r[r.iloc[:, 2] < 93130000].iloc[:, 2].max()

93000950.0

In [208]:
# re["sign"] = np.where(re["cum_volume"] <= re["max_cum_volume_ss"], "delay", "normal")
re[(re["sign"] == "normal") & (re["TransactTime"] < 93130000)]["TransactTime"].max()

93000950.0

In [186]:
r["TransactTime_x"].iloc[:, 0].describe([0.16, 0.17, 0.172, 0.174, 0.176, 0.178, 0.162, 0.164, 0.166, 0.168, 0.169])["16%"]

93000950.0

In [137]:
kk[kk["delayed count"]/ kk["total count"] > 0.8]

,SecurityID,delayed count,total count,min,25%,50%,75%,max
88,600112,414,513,93001020.0,102258530.0,130621340.0,134844110.0,145654670.0
158,600193,273,317,93000660.0,100215040.0,111235120.0,135043040.0,145632460.0
194,600234,202,240,93001710.0,100177117.5,105488550.0,134089977.5,145633680.0
249,600301,221,258,93001040.0,94941050.0,105353610.0,133332560.0,145408310.0
320,600385,228,266,93000970.0,95529930.0,104855620.0,131067332.5,145516120.0
348,600423,217,247,93000730.0,101228380.0,112859250.0,134737970.0,145648930.0
493,600608,326,402,93001860.0,100069185.0,107980085.0,135115712.5,145613520.0
596,600725,273,314,93003120.0,100100440.0,105530270.0,134304620.0,145648090.0
724,600870,285,322,93001020.0,100550200.0,130004860.0,140207530.0,145552010.0
942,601798,195,213,93000500.0,100035360.0,112834120.0,134402560.0,144637540.0


In [138]:
test[test["cum_volume"] <= test["max_cum_volume_ss"]]["TransactTime"].describe([0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                                                                               0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99])

,ApplSeqNum,BidApplSeqNum,ExecType,OfferApplSeqNum,SecurityID,TradeMoney,TradePrice,TradeQty,TransactTime,clockAtArrival,cum_volume,max_cum_volume_ss,sequenceNo
1,27170.0,155362.0,F,148738.0,601798,135460000.0,52100.0,2600.0,93000500.0,1.585013e+15,3300.0,4900.0,2518919
5,31006.0,155362.0,F,166856.0,601798,73080000.0,52200.0,1400.0,93001260.0,1.585013e+15,4700.0,5300.0,2998096
6,31007.0,156432.0,F,166856.0,601798,10420000.0,52100.0,200.0,93001260.0,1.585013e+15,4900.0,5300.0,2998097
8,39669.0,192390.0,F,166856.0,601798,10400000.0,52000.0,200.0,93003060.0,1.585013e+15,5100.0,5300.0,3900391
10,45790.0,209400.0,F,166856.0,601798,10400000.0,52000.0,200.0,93004470.0,1.585013e+15,5300.0,5300.0,4137220
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551,3058517.0,4532237.0,F,5194990.0,601798,67340000.0,51800.0,1300.0,144231240.0,1.585032e+15,504253.0,507253.0,130163342
1552,3058518.0,4745698.0,F,5194990.0,601798,20720000.0,51800.0,400.0,144231240.0,1.585032e+15,504653.0,507253.0,130163343
1553,3058519.0,4814433.0,F,5194990.0,601798,134680000.0,51800.0,2600.0,144231240.0,1.585032e+15,507253.0,507253.0,130163344
1557,3063202.0,5202412.0,F,5194990.0,601798,186480000.0,51800.0,3600.0,144253360.0,1.585032e+15,510853.0,510853.0,130363600


In [30]:
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import Formatter
from bisect import bisect_left, bisect_right

F1 = open("\\\\192.168.10.30\\Kevin_zhenyu\\cffex\\IC1906.pkl", 'rb')
kk = pickle.load(F1)
kk

,date,time,ID,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,...,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,settlePrice,dayToExpiry,contractStatus
0,20190301,2019-03-01 09:29:00.200,IC1906,6,6.057600e+06,5048.0,5045.0,5044.0,5035.2,5032.0,...,5057.8,12,4,1,1,1,5048.0,0.00,112,3
1,20190301,2019-03-01 09:30:00.200,IC1906,22,2.221280e+07,5050.0,5049.6,5049.0,5048.8,5048.2,...,5065.0,1,1,4,2,10,5048.0,0.00,112,3
2,20190301,2019-03-01 09:30:00.700,IC1906,26,2.625520e+07,5056.0,5050.0,5049.8,5049.6,5049.0,...,5058.8,1,1,1,1,1,5048.0,0.00,112,3
3,20190301,2019-03-01 09:30:01.200,IC1906,26,2.625520e+07,5056.0,5054.4,5050.2,5050.0,5049.8,...,5060.4,1,1,2,1,4,5048.0,0.00,112,3
4,20190301,2019-03-01 09:30:01.700,IC1906,29,2.928620e+07,5050.2,5050.2,5050.0,5049.8,5049.0,...,5060.0,1,1,1,2,2,5048.0,0.00,112,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907205,20190621,2019-06-21 14:59:58.200,IC1906,20292,2.039406e+10,5012.0,5011.8,5011.6,5011.2,5011.0,...,5013.4,17,22,4,5,12,4980.0,0.00,0,1
1907206,20190621,2019-06-21 14:59:59.200,IC1906,20296,2.039806e+10,5012.0,5011.8,5011.6,5011.2,5011.0,...,5013.4,13,22,4,5,12,4980.0,0.00,0,1
1907207,20190621,2019-06-21 14:59:59.700,IC1906,20306,2.040809e+10,5012.0,5011.8,5011.6,5011.2,5011.0,...,5013.4,3,22,4,5,12,4980.0,0.00,0,1
1907208,20190621,2019-06-21 15:00:00.200,IC1906,20306,2.040809e+10,5012.0,5011.8,5011.6,5011.2,5011.0,...,5013.4,3,22,4,5,12,4980.0,0.00,0,1


In [27]:
kk[kk["contractStatus"] == 3].groupby('date')['contractStatus'].first()

date
20191021    3
20191022    3
20191023    3
20191024    3
20191025    3
20191028    3
20191029    3
20191030    3
20191031    3
20191101    3
20191104    3
20191105    3
20191106    3
20191107    3
20191108    3
20191111    3
20191112    3
20191113    3
20191114    3
20191115    3
20191118    3
20191119    3
20191120    3
20191121    3
20191122    3
20191125    3
Name: contractStatus, dtype: int64

In [24]:
import datetime
datetime.datetime.strptime("20200320", "%Y%m%d")  - datetime.datetime.strptime("20191125", "%Y%m%d") 

datetime.timedelta(days=116)